# Classification Neural Network

In [ ]:
%matplotlib inline

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import pandas as pd
import copy

In [ ]:
input_data = pd.read_csv("./al_asos_jul_23_v2.csv")

In [ ]:
input_data_renamed_clouds = copy.deepcopy(input_data)
all_cloud_vals = input_data_renamed_clouds['skyc1']
all_cloud_vals.fillna('CLR')
input_data_renamed_clouds['skyc1'] = all_cloud_vals
input_data_renamed_clouds

In [ ]:
input_data_renamed_clouds['raining_status'] = ''
input_data_renamed_clouds['raining_status_int'] = np.empty((len(input_data_renamed_clouds)), dtype=np.int8)

In [ ]:
input_data_renamed_clouds.loc[ input_data_renamed_clouds['p01i'] <= 0.0001, 'raining_status'] = 'TRACE'
input_data_renamed_clouds.loc[ input_data_renamed_clouds['p01i'] <= 0.0001, 'raining_status_int'] = 1

input_data_renamed_clouds.loc[ input_data_renamed_clouds['p01i'] == 0.0, 'raining_status'] = 'DRY'
input_data_renamed_clouds.loc[ input_data_renamed_clouds['p01i'] == 0.0, 'raining_status_int'] = 0

input_data_renamed_clouds.loc[ input_data_renamed_clouds['p01i'] > 0.0001, 'raining_status'] = 'RAIN'
input_data_renamed_clouds.loc[ input_data_renamed_clouds['p01i'] > 0.0001, 'raining_status_int'] = 0


In [ ]:
# Let's try adding different layers
input1 = keras.layers.Input(shape=(1, ))
dense1 = keras.layers.Dense(10, input_dim=1, activation=keras.activations.sigmoid)(input1)

# 3 outputs, as we will predict the likelyhood of each classification
output = keras.layers.Dense(3, activation=keras.activations.relu, use_bias=True)(dense1)
model10 = keras.models.Model(inputs=[input1], outputs=[output])
# need to use a different loss function
model10.compile(optimizer=keras.optimizers.legacy.Adam(learning_rate=0.0001, clipnorm=0.001), 
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
model10.summary()

In [ ]:
model10.fit(input_data_renamed_clouds['p01i'], input_data_renamed_clouds['raining_status_int'].values, epochs=4)